In [1]:
from bs4 import BeautifulSoup
import urllib
from urllib.request import urlopen
import re
import pandas as pd
import requests
from requests import get

# Pulling all links off of the search pages (up to 20) and putting them in a dataframe to scrape.
Will use 20 for the sake of trying it out...change it later.

In [2]:
url_template="http://www.indeed.com/jobs?q=%22Data+Scientist%22&l=USA&start={}"
max_results=20
Linkdf=[]

for start in range(0, max_results, 10):
    url=url_template.format(start)
    html=requests.get(url)
    soup=BeautifulSoup(html.content,'html.parser', from_encoding="utf-8")
    
    #for each in soup.find_all(a_="href"):
    page_links=soup.find_all('a',{'href':re.compile("/rc/")})
    for items in page_links:
        Linkdf.append(items['href'])   

In [3]:
#Sanity Check
len(Linkdf)
#print(Linkdf)

14

In [4]:
#This code allows the code to display the full website instead of truncating
pd.set_option('display.max_colwidth',-1)

#Moving it to a data frame
data = {'links':Linkdf}
df = pd.DataFrame(data, columns=['links'])

#append indeed.com to the front of each
df['Web'] = 'https://www.indeed.com'
df['URL'] = df.Web.str.cat(df.links)

#pull out just a list of the websites.
websites=list(df['URL'])
print(websites)

['https://www.indeed.com/rc/clk?jk=1e686166d30fd118&fccid=de71a49b535e21cb&vjs=3', 'https://www.indeed.com/rc/clk?jk=267e7cf7de2959c7&fccid=de71a49b535e21cb&vjs=3', 'https://www.indeed.com/rc/clk?jk=020c5d3fcc757260&fccid=79462bd4e96de970&vjs=3', 'https://www.indeed.com/rc/clk?jk=95f7f6a9d224911d&fccid=3c371b712ace44cf&vjs=3', 'https://www.indeed.com/rc/clk?jk=1942a17d2bd166e1&fccid=ae0c894528aa6eee&vjs=3', 'https://www.indeed.com/rc/clk?jk=887b613c8fe29cec&fccid=600ab51dbaf8fbb5&vjs=3', 'https://www.indeed.com/rc/clk?jk=83e3f665b22e5695&fccid=f01127702f08b4dc&vjs=3', 'https://www.indeed.com/rc/clk?jk=11baba484db23433&fccid=734cb5a01ee60f80&vjs=3', 'https://www.indeed.com/rc/clk?jk=1eb7e96ae99c871d&fccid=1639254ea84748b5&vjs=3', 'https://www.indeed.com/rc/clk?jk=a327d4b7bde3ff16&fccid=ac3ca395d43dcd7d&vjs=3', 'https://www.indeed.com/rc/clk?jk=42e7131e05fbcde9&fccid=877cae5cde4da80e&vjs=3', 'https://www.indeed.com/rc/clk?jk=8c9277695be5f139&fccid=b2d899f595c435be&vjs=3', 'https://www.in

# Looping through websites....

In [5]:
#Loop

Title=[]
Descriptions=[]
Location=[]

for url in websites:
    response=get(url)
    soup=BeautifulSoup(response.text,'html.parser')
    
    description_containers= soup.find(class_='jobsearch-jobDescriptionText')
    title_containers=soup.find('h3')
    try:
        location_containers=soup.find('',{'class':'jobsearch-CompanyInfoWithoutHeaderImage'}).find_all('div')[-1]
    except:
        location_containers='None Found'
    
    job_descriptions=str(description_containers)
    job_title=str(title_containers.text)
    locations=str(location_containers.text)
    
    Descriptions.append(job_descriptions)
    Title.append(job_title)
    Location.append(locations)

# THIS GIVES US WHAT WE NEED BUT PUTS IT ALL IN THE SAME COLUMN
## Right now, the code pulls Qualifications, Skills, Responsibilities, Education, and Requirements into "Qualifications".  We want to be able to have a each one as it's own column in a dataframe. (right now it's only pulling 20 sites max, so there are a lot of 'None Found'. When we pull the larger chunk, there will be a lot more!). :-) 

In [6]:
#THIS GIVES US WHAT WE NEED BUT PUTS IT ALL IN THE SAME COLUMN
Search_Values = ['Qualifications', 'Skills', 'Responsibilities', 'Education', 'Requirement']
i=0

Qualifications = []

for n in range(0, len(Descriptions)):
    for values in Search_Values :
        soup=BeautifulSoup(Descriptions[n],'lxml')

        Qualifications_tag = soup.find('b', text=re.compile(values))
        try:
            ul_tag = Qualifications_tag.find_next('ul')
            Qualification=ul_tag.text
            Qualifications.append({i,Qualification})
        except AttributeError: 
            Qualification="None Found"
            Qualifications.append({i,Qualification})
    i=i+1

In [8]:
Qualifications

[{0, 'None Found'},
 {0, 'None Found'},
 {0,
  'Ability to look at things differently, debug, troubleshoot, design and implement solutions to complex technical issues.\nStrong technical and analytical abilities, a knack for driving impact and growth, and experience with a programming/scripting in a language such as Java or Python.\nBasic understanding of statistical programming in a language such as R, Python, or SAS, SPSS, MATLAB.\nBasic understanding of Cloud (AWS, Azure, etc.)\nExcellent verbal and written communication skills.\nWork or internship experience using data science tools in a corporate environment.\nInterest in, understanding of, or experience with Design Thinking and Agile Development Methodologies\nWillingness to travel up to 100% of the time.'},
 {0, 'None Found'},
 {0, 'None Found'},
 {1, 'None Found'},
 {1, 'None Found'},
 {'\nYou will implement and validate predictive models as well as create and maintain statistical models with a focus on big data.\nYou will be ex

# THIS WAS THE LAST ONE WE WERE WORKING ON - DOESN'T QUITE DO WHAT WE WANT YET.

In [9]:
#THIS WAS THE LAST ONE WE WERE WORKING ON - DOESN'T QUITE DO WHAT WE WANT YET.
Search_Values = ['Qualifications', 'Skills', 'Responsibilities', 'Education', 'Requirement']
i=0

Qualifications = []
Skills = []
Responsibilities = []
Education = []
Requirements = []
Master = []

for n in range(0, len(Descriptions)):
    for values in Search_Values :
        soup=BeautifulSoup(Descriptions[n],'lxml')

        Qualifications_tag = soup.find('b', text=re.compile(values))
        temp=[]
        try:
            ul_tag = Qualifications_tag.find_next('ul')
            Qualification=ul_tag.text
            temp.append({i,Qualification})
            df = pd.DataFrame(temp)
        except AttributeError: 
            Qualification="None Found"
            temp.append({i,Qualification})
            df = pd.DataFrame(temp)
        #Master[m].append(df)
        #m = m +1
   # Master[m].append(Skills)
    i=i+1        

# Below here works, just need to get the above working.

In [13]:
#Full descriptions in dataframe
df_descriptions = pd.DataFrame({'Full_Description': Descriptions})
df_descriptions

Full_Description
0   <div class="jobsearch-jobDescriptionText" id="jobDescriptionText"><div><b>Introduction</b><br/>\nAs a Data Scientist at IBM, you will help transform our clients’ data into tangible business value by analyzing information, communicating outcomes and collaborating on product development. Work with Best in Class open source and visual tools, along with the most flexible and scalable deployment options. Whether it’s investigating patient trends or weather patterns, you will work to solve real world problems for the industries transforming how we live.<br/>\n<br/>\n<b>Your Role and Responsibilities</b><br/>\nImplement and validate predictive and prescriptive models, create and maintain statistical models with a focus on big data.<br/>\nIncorporate a variety of statistical and machine learning techniques in your projects.<br/>\nWrite programs to cleanse and integrate data in an efficient and reusable manner.<br/>\nUse leading edge and open-source tools such as Python, R, and TensorFlow, combined with IBM tools and our AI application suites.<br/>\nWork in an Agile, collaborative environment, partnering with other scientists, engineers, consultants and database administrators of all backgrounds and disciplines to bring analytical rigor and statistical methods to the challenges of predicting behaviors.<br/>\nCommunicate with internal and external clients to understand and define business needs and appropriate modelling techniques to provide analytical solutions.<br/>\nEvaluate modelling results and communicate the results to technical and non-technical audiences.<br/>\n<br/>\n<br/>\n<b>Required Professional and Technical Expertise</b><br/>\n<ul><li>Ability to look at things differently, debug, troubleshoot, design and implement solutions to complex technical issues.</li><li>\nStrong technical and analytical abilities, a knack for driving impact and growth, and experience with a programming/scripting in a language such as Java or Python.</li><li>\nBasic understanding of statistical programming in a language such as R, Python, or SAS, SPSS, MATLAB.</li><li>\nBasic understanding of Cloud (AWS, Azure, etc.)</li><li>\nExcellent verbal and written communication skills.\n</li><li>Work or internship experience using data science tools in a corporate environment.</li><li>\nInterest in, understanding of, or experience with Design Thinking and Agile Development Methodologies</li><li>\nWillingness to travel up to 100% of the time.</li></ul><b>\nMajors:</b><br/>\nComputer Science, Data Science, Applied Mathematics, Statistics, Cognitive Science, Artificial Intelligence, Business Intelligence, Operations Research, Engineering<br/>\n<br/>\nLocations:<br/>\nCambridge, MA<br/>\nChicago, IL<br/>\nDallas, TX<br/>\nHouston, TX<br/>\nNew York, NY<br/>\nSan Francisco, CA<br/>\nWashington DC<br/>\n<br/>\n<br/>\n<b>Preferred Professional and Technical Expertise</b><br/>\nNA<br/>\n<br/>\n<b>About Business Unit</b><br/>\nIBM Services is a team of business, strategy and technology consultants that design, build, and run foundational systems and services that is the backbone of the world's economy. IBM Services partners with the world's leading companies in over 170 countries to build smarter businesses by reimagining and reinventing through technology, with its outcome-focused methodologies, industry-leading portfolio and world class research and operations expertise leading to results-driven innovation and enduring excellence.<br/>\n<br/>\n<b>Your Life @ IBM</b><br/>\nWhat matters to you when you’re looking for your next career challenge?<br/>\n<br/>\nMaybe you want to get involved in work that really changes the world? What about somewhere with incredible and diverse career and development opportunities – where you can truly discover your passion? Are you looking for a culture of openness, collaboration and trust – where everyone has a voice? What about all of these? If so, then IBM could be your next career challenge. Join us, not to do s

In [14]:
df_title=pd.DataFrame({'Job_Title':Title})
df_title

,Job_Title
0,Entry Level - Associate Data Scientist
1,Entry Level Data Scientist
2,"Data Scientist Intern, Boston"
3,Data Scientist
4,Jr. Data Scientist
5,Jr. Data Scientist
6,Data Scientist
7,Full Time Opportunities for Students or Recent Graduates: Data & Applied Sciences
8,"Data Scientist, Analytics, Intern"
9,"Data Scientist, Intermediate"


In [15]:
df_locations=pd.DataFrame({'Location':Location})
df_locations

,Location
0,United States
1,United States
2,"Boston, MA"
3,"Englewood, CO 80113"
4,"New York, NY 10036"
5,"Glendale, CA"
6,"New York, NY"
7,"Redmond, WA"
8,"New York, NY 10017"
9,"Charlottesville, VA"


In [16]:
FullDataSet = pd.concat([df_title,df_locations, df_descriptions], axis=1)
FullDataSet

Job_Title  \
0   Entry Level - Associate Data Scientist                                              
1   Entry Level Data Scientist                                                          
2   Data Scientist Intern, Boston                                                       
3   Data Scientist                                                                      
4   Jr. Data Scientist                                                                  
5   Jr. Data Scientist                                                                  
6   Data Scientist                                                                      
7   Full Time Opportunities for Students or Recent Graduates: Data & Applied Sciences   
8   Data Scientist, Analytics, Intern                                                   
9   Data Scientist, Intermediate                                                        
10  Data Scientist                                                                      
11  Data Scientist - Entry Level                                                        
12  Data Scientist                                                                      
13  2020 Intern - Data Scientist Intern/Coop (EO&S)                                     

                      Location  \
0   United States                
1   United States                
2   Boston, MA                   
3   Englewood, CO 80113          
4   New York, NY 10036           
5   Glendale, CA                 
6   New York, NY                 
7   Redmond, WA                  
8   New York, NY 10017           
9   Charlottesville, VA          
10  New York, NY                 
11  Washington, DC               
12  Glendale, CA                 
13  Research Triangle Park, NC   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     